In [1]:
#Import necessary packages
import math
import numpy as np
np.set_printoptions(suppress=True,
                    formatter={'all': lambda x: '%5.3f' % x})
import pandas as pd
from scipy.optimize import brute, fmin, minimize
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

#Install import_ipynb so we can read dependenies as jupyter notebooks
!pip install ipynb

#Import dependencies
from ipynb.fs.full.FFT_option_valuation import M76_call_value_FFT
from ipynb.fs.full.CIR_calibration import CIR_calibration , r_list
from ipynb.fs.full.CIR_zcb_valuation import B

In [2]:
#Import CSV file with option data as a panda dataframe
raw = pd.read_csv('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/option_data.csv')

#Convert variables Maturity and Date to pandas datetime objects
raw['Maturity']= pd.to_datetime(raw['Maturity'], format='%Y-%m-%d') 
raw['Date']= pd.to_datetime(raw['Date'], format='%Y-%m-%d')

In [3]:
#Create a copy so we do not have to reload the data set if we start over
data = raw.copy()
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Maturity  159 non-null    datetime64[ns]
 1   Strike    159 non-null    float64       
 2   Call      159 non-null    float64       
 3   Date      159 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2)
memory usage: 5.1 KB


,Maturity,Strike,Call,Date
0,2020-10-29,205.0,21.500,2020-10-26
1,2020-10-29,210.0,16.375,2020-10-26
2,2020-10-29,215.0,11.500,2020-10-26
3,2020-10-29,220.0,6.950,2020-10-26
4,2020-10-29,225.0,3.050,2020-10-26
...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26
155,2021-12-17,270.0,5.250,2020-10-26
156,2021-12-17,290.0,2.450,2020-10-26
157,2021-12-17,310.0,1.100,2020-10-26


In [4]:
#Create a variable, T, that shows how much, as a fraction of a year, that is left until maturity
data['diff'] = data['Maturity'] - data['Date']                     #number of days left to maturity
data['T']=(data['diff'].astype('timedelta64[D]').astype(int))/365  

#Drop unnecessary varibles
data.drop(['diff'], axis = 1, inplace=True)
options = data
options

,Maturity,Strike,Call,Date,T
0,2020-10-29,205.0,21.500,2020-10-26,0.008219
1,2020-10-29,210.0,16.375,2020-10-26,0.008219
2,2020-10-29,215.0,11.500,2020-10-26,0.008219
3,2020-10-29,220.0,6.950,2020-10-26,0.008219
4,2020-10-29,225.0,3.050,2020-10-26,0.008219
...,...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26,1.142466
155,2021-12-17,270.0,5.250,2020-10-26,1.142466
156,2021-12-17,290.0,2.450,2020-10-26,1.142466
157,2021-12-17,310.0,1.100,2020-10-26,1.142466


In [5]:
#Initial short rate (Eonia 3.12.2020)
r0 = r_list[0]  

#Calibrate Short Rate Model to get calibrated estimates of CIR parameters used for zcb valuation
kappa_r, theta_r, sigma_r = CIR_calibration()

r = []
for row, option in options.iterrows():
    B0T = B([kappa_r, theta_r, sigma_r, r0, option['T']])
    r.append(-math.log(B0T) / option['T'])
options['r'] = r
options

Optimization terminated successfully.
         Current function value: 0.000020
         Iterations: 201
         Function evaluations: 382


,Maturity,Strike,Call,Date,T,r
0,2020-10-29,205.0,21.500,2020-10-26,0.008219,-0.004668
1,2020-10-29,210.0,16.375,2020-10-26,0.008219,-0.004668
2,2020-10-29,215.0,11.500,2020-10-26,0.008219,-0.004668
3,2020-10-29,220.0,6.950,2020-10-26,0.008219,-0.004668
4,2020-10-29,225.0,3.050,2020-10-26,0.008219,-0.004668
...,...,...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26,1.142466,0.000189
155,2021-12-17,270.0,5.250,2020-10-26,1.142466,0.000189
156,2021-12-17,290.0,2.450,2020-10-26,1.142466,0.000189
157,2021-12-17,310.0,1.100,2020-10-26,1.142466,0.000189


In [6]:
##################################################################################################################
################################################ Preliminaries ###################################################

#Define Stock price 
S0 = 226

#Slice the dataframe
#Default option selects all options in the csv file

#Choose only options that are within 2% of ATM 
#tol = 0.02  # percent ITM/OTM options
#options = [(np.abs(options['Strike'] - S0) / S0) < tol] 

#Choose specific strike prices 
# options = options[options['Strike'].isin([180, 200, 220, 240, 260])]

##################################################################################################################
######################################## Loss Function for the calibration #######################################

i = 0                                                              #Initialize iteration counter
min_MSE = 500                                                      #Initial min_MSE value
def M76_error_function_FFT(p0):
    ''' Error Function for parameter calibration in M76 Model via
    Carr-Madan (1999) FFT approach.

    Parameters
    ==========
    sigma: float
        volatility factor in diffusion term
    lamb: float
        jump intensity
    mu: float
        expected jump size
    delta: float
        standard deviation of jump

    Returns
    =======
    RMSE: float
        root mean squared error
    '''
    global i, min_MSE                              #Set global variables                           
    sigma, lamb, mu, delta = p0                     #Define initial values of the parameter vector p0
    if sigma < 0.0 or delta < 0.0 or lamb < 0.0: 
        return 500.0                                #Make sure that sigma, delta, lambda > 0 
    
    se = []                                                     #Initialize the vector to store squared errors
    for row, option in options.iterrows():                      #Run for loop over all columns in each row
        model_value = M76_call_value_FFT(S0, option['Strike'], option['T'], option['r'], sigma, lamb, mu, delta)
        se.append((model_value - option['Call']) ** 2)          #Add a the next squared error to the array 'se'
    MSE = sum(se) / len(se)                                     #Calculate the MSE
    min_MSE = min(min_MSE, MSE)               #Define min_MSE as the smallest of current min_MSE and new MSE
    if i % 25 == 0:                              #Print the parameter vector, MSE and min_MSE every 25th run
        print('%4d |' % i, np.array(p0), '| %7.3f | %7.3f' % (MSE, min_MSE))
    i += 1
    return MSE

##################################################################################################################
####################### Calibration of Merton (1976) parameters sigma, lambda, mu, delta #########################

def M76_calibration_full_FFT():
    ''' Calibrates M76 stochastic volatility model to market quotes. '''
    # first run with brute force
    # (scan sensible regions)
    p0 = brute(M76_error_function_FFT, 
               ((0.075, 0.201, 0.025), #sigma
                (0.10, 0.401, 0.1),    #
                (-0.5, 0.01, 0.1),
                (0.10, 0.301, 0.1)), 
                finish=None)

    # second run with local, convex minimization
    # (dig deeper where promising)
    opt = fmin(M76_error_function_FFT, p0, 
               xtol=0.001, ftol=0.001,
               maxiter=1000, maxfun=1000)
    np.save('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/opt_jump', np.array(opt))
    return opt

##################################################################################################################
################### Calculation of Merton (1976) option values with stochastic interest rates  ###################

def M76_calculate_model_values_FFT(p0):
    ''' Calculates all model values given parameter vector p0. '''
    sigma, lamb, mu, delta = opt 
    values = []
    for row, option in options.iterrows():
        model_value = M76_call_value_FFT(S0, option['Strike'], option['T'], option['r'], sigma, lamb, mu, delta)
        values.append(model_value)
    return np.array(values) 

##################################################################################################################
################################################# Run calibration ################################################

if __name__ == '__main__': 
    
    #Run and print the calibration
    opt = M76_calibration_full_FFT()
    print('')
    print("Calibrated parameter vector: ", opt)

    #Create pandas column for Heston (1993) call options values based on calibrated data
    options['Model'] = M76_calculate_model_values_FFT(opt)

   0 | [0.075000 0.100000 -0.500000 0.100000] |   8.857 |   8.857
  25 | [0.075000 0.200000 -0.300000 0.200000] |   8.080 |   5.429
  50 | [0.075000 0.300000 -0.100000 0.300000] |   6.532 |   4.428
  75 | [0.100000 0.100000 -0.400000 0.100000] |   7.642 |   3.651
 100 | [0.100000 0.200000 -0.200000 0.200000] |   7.483 |   3.651
 125 | [0.100000 0.300000 0.000000 0.300000] |   5.149 |   3.227
 150 | [0.125000 0.100000 -0.300000 0.100000] |   6.334 |   2.918
 175 | [0.125000 0.200000 -0.100000 0.200000] |   6.157 |   2.723
 200 | [0.125000 0.400000 -0.500000 0.300000] |   7.184 |   2.270
 225 | [0.150000 0.100000 -0.200000 0.100000] |   4.901 |   2.181
 250 | [0.150000 0.200000 0.000000 0.200000] |   4.253 |   1.943
 275 | [0.150000 0.400000 -0.400000 0.300000] |   4.190 |   1.766
 300 | [0.175000 0.100000 -0.100000 0.100000] |   3.378 |   1.526
 325 | [0.175000 0.300000 -0.500000 0.200000] |   4.231 |   1.398
 350 | [0.175000 0.400000 -0.300000 0.300000] |   2.824 |   1.324
 375 | [0.20

In [7]:
#Drop max.rows option so we can see full dataframe 
pd.set_option('display.max_rows', None)
options

,Maturity,Strike,Call,Date,T,r,Model
0,2020-10-29,205.0,21.500,2020-10-26,0.008219,-0.004668,21.084053
1,2020-10-29,210.0,16.375,2020-10-26,0.008219,-0.004668,16.099880
2,2020-10-29,215.0,11.500,2020-10-26,0.008219,-0.004668,11.116964
3,2020-10-29,220.0,6.950,2020-10-26,0.008219,-0.004668,6.207753
4,2020-10-29,225.0,3.050,2020-10-26,0.008219,-0.004668,2.140486
5,2020-10-29,230.0,0.850,2020-10-26,0.008219,-0.004668,0.309584
6,2020-10-29,235.0,0.360,2020-10-26,0.008219,-0.004668,0.014069
7,2020-11-06,205.0,21.625,2020-10-26,0.030137,-0.004556,21.307393
8,2020-11-06,210.0,17.000,2020-10-26,0.030137,-0.004556,16.383810
9,2020-11-06,215.0,12.500,2020-10-26,0.030137,-0.004556,11.569481
